In [1]:
#Import the important packages
import pandas as pd
from flask import Flask, request, render_template, jsonify
import json
from datetime import datetime, timedelta
import requests
import psycopg2
from psycopg2.extras import execute_values
import os
import sys
from pymongo import MongoClient

In [2]:
#Start the flask
app = Flask(__name__)

In [3]:
#Import all the necessary data files
user_file_path = "/Users/anusha/Downloads/Netflix_Userbase.csv"
user_dataset = pd.read_csv(user_file_path)
user_dataset['last_payment'] = pd.to_datetime(user_dataset['last_payment'], format='%d-%m-%y')
user_dataset['Join_Date'] = pd.to_datetime(user_dataset['Join_Date'], format='%d-%m-%y')
columns_to_keep = ['UserID', 'SubscriptionType', 'Join_Date', 'last_payment']
user_dataset = user_dataset[columns_to_keep]

art_locations = pd.read_csv('/Users/anusha/Desktop/Columbia School/Managing Data/pois.csv')

In [4]:
#Home Screen that brings the user to a welcome page
@app.route('/')
def home():
    return f"Welcome to Craze"

In [5]:
#Use Postgres to understand if the user has paid their subscription
#Create the table for user information
@app.route('/user')
def user_create():
    try:
        conn = psycopg2.connect("dbname='a_2_db' user='postgres' host='localhost' password='123'")
        cur = conn.cursor()
        
        createCmd = """
        CREATE TABLE IF NOT EXISTS Users (
            UserID SERIAL PRIMARY KEY,
            SubscriptionType VARCHAR(255),
            Join_Date DATE,
            last_payment DATE
        );
        """
        cur.execute(createCmd)
        conn.commit()
        
    except Exception as e:
        return str(e)
    finally:
        cur.close()
        conn.close()
    
    return "Fetched data successfully."

#Insert the data from the dataset to the table
def insert_data_from_df(df):
    conn = psycopg2.connect("dbname='a_2_db' user='postgres' host='localhost' password='123'")
    cur = conn.cursor()
    try:
        tuples = [tuple(x) for x in df.to_numpy()]
        cols = ','.join(list(df.columns))
        query = "INSERT INTO Users (%s) VALUES %%s" % cols
        execute_values(cur, query, tuples)
        conn.commit()
    except Exception as e:
        print("Error: ", e)
    finally:
        cur.close()
        conn.close()
        
@app.route("/redirect-internal", methods=["GET"])
def redirect_internal():
    return redirect("/landing", code=302)        

# Call the function
insert_data_from_df(user_dataset)

#Create a function to check user's payment that defaults to 24 if needed
def check_payment_status(user_number = 24):
    try:
        conn = psycopg2.connect("dbname='a_2_db' user='postgres' host='localhost' password='123'")
        cur = conn.cursor()
        
        # Query to get the last payment date for the user
        cur.execute("SELECT last_payment FROM Users WHERE UserID = %s", (user_number,))
        last_payment = cur.fetchone()
        if last_payment is None:
            return "User not found"

        last_payment_date = last_payment[0]  # assuming last_payment is not None
        today = datetime.today().date()
        one_month_timedelta = timedelta(days=30)

        # Calculate if payment is pending
        payment_pending = (today - last_payment_date).days > 30

        return "Cannot continue as payment is pending for your account" if payment_pending else "Welcome back!"
    except Exception as e:
        return str(e)
    finally:
        cur.close()
        conn.close()
        
 #When the user_id is called we return the correct message       
@app.route('/user/<int:user_number>')
def user(user_number):
    message = check_payment_status(user_number)
    return message



Error:  duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (userid)=(1) already exists.



In [6]:
#Running an API from Meetup with PySpark
    #The Meetup API is through GraphSQL therefore cannot be demoed without a third party client easily. 
    #However the following is the code that would call the API based on a user's location choice and provide a list of meetup events in the region specified
@app.route('/search_events', methods=['GET'])
def search_events():
    # Ensure location is provided as a query parameter
    location = request.args.get('location')
    if not location:
        return jsonify({"error": "No location specified"}), 400

    API_KEY = 'u5sjebnqn9upnue9qbbkhhcq67'

    url = f'https://api.meetup.com/find/events?key={API_KEY}&sign=true&photo-host=public&location={location}'

    # Function to make API request
    try:
        response = requests.get(url)
        if response.status_code == 200:
            events_data = response.json()
            events_output = []
            for event in events_data:
                if 'name' in event and 'venue' in event and 'address_1' in event['venue'] and 'local_date' in event:
                    events_output.append({
                        'name': event['name'],
                        'address': event['venue']['address_1'],
                        'date': event['local_date']
                    })
            return jsonify(events_output)
        else:
            return jsonify({"error": "Failed to fetch events"}), 500
    except requests.exceptions.RequestException as e:
        return jsonify({"error": str(e)}), 500

In [7]:
#Use Mongo to parse the POIs in NYC to find arts centers for the user
    #This example takes a dataset pulled from OSM of POIs
@app.route('/arts_locations')
def arts_locations():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['managing_data_db']
    collection = db['locations']

    # Check if the collection is empty
    if collection.count_documents({}) == 0:
        locations_data = art_locations.to_dict(orient='records')
        collection.insert_many(locations_data)

    # Retrieve only the names of the arts centers
    arts_centers = collection.find({"fclass": "arts_centre"}, {"name": 1, "_id": 0})

    results = [center['name'] for center in arts_centers]  # Extracting names from the query results

    return jsonify(results)

In [ ]:
app.run(host='localhost', port=5032)

if __name__ == "__main__":  
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5032
Press CTRL+C to quit
127.0.0.1 - - [25/Apr/2024 02:42:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 02:42:59] "GET /user HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 02:43:10] "GET /user/24 HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 02:43:33] "GET /search_events HTTP/1.1" 400 -
127.0.0.1 - - [25/Apr/2024 02:44:03] "GET /arts_locations HTTP/1.1" 200 -
